<a href="https://colab.research.google.com/github/kim-kidong/hello-world/blob/master/%EC%A3%BC%EC%8B%9D%EA%B0%80%EA%B2%A9%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 주식가격 예측(LSTM)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout,Dense,Activation
import datetime

Using TensorFlow backend.


# Data입력

In [2]:
data = pd.read_csv("005930.KS.csv")
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-10-17,44150.0,44500.0,44000.0,44150.0,43742.46094,8306193.0
1,2018-10-18,43950.0,44450.0,43700.0,44050.0,43643.38281,8347725.0
2,2018-10-19,43900.0,44150.0,43450.0,43900.0,43494.76953,7842520.0
3,2018-10-22,43450.0,43950.0,43200.0,43550.0,43148.00000,8357648.0
4,2018-10-23,43300.0,43450.0,42550.0,43050.0,42652.61328,9531231.0


# Compute Mid Price

In [0]:
high_prices = data['High'].values
low_prices = data['Low'].values
mid_prices = (high_prices + low_prices)/2

#Create Windows

In [0]:
seq_len = 50
sequence_length = seq_len + 1


result = []
for index in range(len(mid_prices) - sequence_length):
  result.append(mid_prices[index:index + sequence_length])

# Normalize Data

In [6]:
normalized_data = []
for window in result:
  normalized_window = [((float(p)/float(window[0]))-1) for p in window]
  normalized_data.append(normalized_window)
  
  
result = np.array(normalized_data)


row = int(round(result.shape[0]*0.9))
train = result[:row, :]
np.random.shuffle(train)

x_train = train[:, :-1]
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
y_train = train[:, :-1]

x_test = result[row:, :-1]
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1],1))
y_test = result[row:, :-1]

x_train.shape, x_test.shape

((176, 50, 1), (19, 50, 1))

# Build a Model

In [8]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(50,1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='rmsprop')
model.summary()





Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 50)            10400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________


# Traing

In [9]:
model.fit(x_train, y_train,
  validation_data=(x_test, y_test),
  batch_size=10, epochs=20)

ValueError: ignored

# Predition

In [10]:
pred = model.predict(x_test)

fig = plt.figure(facecolor='white')
ax = fig.subplot(111)
ax.plot(y_test, label='True')
ax.plot(pred, label='prediction')
ax.legend()
plt.show()

AttributeError: ignored

<Figure size 432x288 with 0 Axes>